In [80]:
import pandas as pd
import requests
import os
import re

In [81]:
month =  "03"
year = "2021"
date = month + year
file_url = f"https://www.huduser.gov/portal/datasets/usps/ZIP_COUNTY_{date}.xlsx"
df = pd.read_excel(file_url, usecols="A:D", dtype=str)
df.rename(columns={"COUNTY":"ST_CTY_FIPS"}, inplace=True)
df.head()

,ZIP,ST_CTY_FIPS,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE
0,00501,36103,HOLTSVILLE,NY
1,00601,72113,ADJUNTAS,PR
2,00601,72001,ADJUNTAS,PR
3,00602,72005,AGUADA,PR
4,00602,72003,AGUADA,PR


In [82]:
df["ZIP"] = df["ZIP"].apply(lambda x: x.zfill(5))
df["ZIP"].head()

0    00501
1    00601
2    00601
3    00602
4    00602
Name: ZIP, dtype: object

In [83]:
df["YEAR"] = year
df["MONTH"] = month
third_col = df.pop("YEAR")
fourth_col = df.pop("MONTH")

df.insert(2, "YEAR", third_col)
df.insert(3, "MONTH", fourth_col)
df.head()

,ZIP,ST_CTY_FIPS,YEAR,MONTH,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE
0,00501,36103,2021,03,HOLTSVILLE,NY
1,00601,72113,2021,03,ADJUNTAS,PR
2,00601,72001,2021,03,ADJUNTAS,PR
3,00602,72005,2021,03,AGUADA,PR
4,00602,72003,2021,03,AGUADA,PR


In [84]:
fips_url = "https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt"
fips_r = requests.get(fips_url)

MAIN_DIR =  "D:\\Code\\PYTHON\\FIPS_Address_Matching"
fips_path = MAIN_DIR + "\\data\\gov_fips.csv"

with open(fips_path, 'wb') as url:
    for chunk in fips_r.iter_content(chunk_size=1024):
        if chunk:
            url.write(chunk)

In [85]:
with open(fips_path, 'r') as file:
    raw_lines = file.readlines()
    start_index = raw_lines.index('  FIPS code        name\n')
    # We want to drop the first line which is the fips_code and name, as well as the second line which is just blank space
    cleaning_lines = raw_lines[start_index+2:]
    df2 = pd.DataFrame(columns=["ST_CTY_FIPS", "Name"])
    code_list = []
    name_list = []

    for line in cleaning_lines:
        
        leading_trailing_space = re.compile(r"^\s+|\s+$")
        line = leading_trailing_space.sub('',line)

        parenthesis_after = re.compile(r"\(.*$")
        line = parenthesis_after.sub('', line)
        
        extra_whitespace = re.compile(r"\s+")
        line = extra_whitespace.sub(' ',line)
        
        line_list = line.split(' ',1)
        code_list.append(line_list[0])
        name_list.append(line_list[1])

    concat_dict = {"ST_CTY_FIPS": code_list, "Name": name_list}
    df2 = pd.concat([pd.Series(value, name=key) for key, value in concat_dict.items()], axis=1)
    
fips_path = MAIN_DIR + "\\data\\county_fips.csv"
df2.to_csv(fips_path, index=False)
df2.head()

,ST_CTY_FIPS,Name
0,01000,Alabama
1,01001,Autauga County
2,01003,Baldwin County
3,01005,Barbour County
4,01007,Bibb County


In [87]:
merged_df = df2.merge(df)
merged_df.head()

,ST_CTY_FIPS,Name,ZIP,YEAR,MONTH,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE
0,01001,Autauga County,36003,2021,03,AUTAUGAVILLE,AL
1,01001,Autauga County,36006,2021,03,BILLINGSLEY,AL
2,01001,Autauga County,36008,2021,03,BOOTH,AL
3,01001,Autauga County,36022,2021,03,DEATSVILLE,AL
4,01001,Autauga County,36051,2021,03,MARBURY,AL
